In [14]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd

import sys
sys.path.append('..')
import time

from src.beam.parallel import BeamTask, BeamParallel, parallel, task
from src.beam.utils import collate_chunks, divide_chunks
from src.beam.data import BeamData
from mp_functions import func

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Beam Data

In [15]:
n = 100
m = 100
k = 4

# dfs = {'a': {rand_column(): pd.DataFrame(index=np.random.permutation(np.arange(n)),
#                     data=np.random.randn(n, m), columns=[rand_column() for _ in range(m)]) for _ in range(k)},
#        'b': [pd.DataFrame(index=np.random.permutation(np.arange(n)),
#                     data=np.random.randn(n, m), columns=[rand_column() for _ in range(m)]) for _ in range(k)]}

dfs = {'a': torch.randn(200, m), 'b': torch.randn(300, m), 'c': torch.randn(400, m)}

# dfs = pd.DataFrame(index=np.random.permutation(np.arange(n)),
#                     data=np.random.randn(n, m), columns=[rand_column() for _ in range(m)])

path = '/tmp/sandbox/bd'
bd = BeamData(dfs, path=path)

In [16]:
l = list(iter(bd))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_81372/53212255.py", line 1, in <cell line: 1>
    l = list(iter(bd))
  File "/home/elad/docker/beamds/notebooks/../src/beam/data.py", line 1404, in __iter__
    yield k, BeamData(v, lazy=self.lazy, columns=self.columns, index=index, label=label)
  File "/home/elad/docker/beamds/notebooks/../src/beam/data.py", line 79, in __init__
    assert len(list(filter(lambda x: x is not None, [data, path]))), \
AssertionError: Requires ether data, root_path or all_paths to be not None

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/opt/conda/lib/python3.8/site-p

In [6]:
x = bd.stack

In [10]:
x.dtypes

'array.tensor.float'

In [5]:
bd.store()

In [6]:
be = BeamData.from_path(path)

In [7]:
be.cache()

In [18]:
be['x'] = np.arange(100)

In [26]:
be['a', :200, :10]

BeamData: 
  path: 
  None 
  params: 
  orientation: simple | lazy: True | stored: False | cached: True | device: cpu | objects_type: tensor | quick_getitem: False | has_index: True | has_label: False 
  keys: 
  slice(None, 10, None) 
  sizes:
  8000 
  shapes:
  torch.Size([200, 10]) 
  types:
  array.tensor.float 

In [2]:
df = pd.DataFrame(np.random.randn(10000, 500))

In [3]:
dfs = list(divide_chunks(df, n_chunks=20))

In [4]:
def func(df):
    time.sleep(10)
    return df.agg([np.sum, 'mean'], axis=1)

@task
def func2(df):
    time.sleep(10)
    return df.agg([np.sum, 'mean'], axis=1)

In [5]:
%time dfo = parallel([func2(dfi) for _, dfi in dfs], method='ray', workers=4, reduce=True, runtime_env={"working_dir": ".." })

2023-01-18 12:08:41,357	INFO worker.py:1333 -- Connecting to existing Ray cluster at address: 172.17.0.2:46610...
2023-01-18 12:08:41,366	INFO worker.py:1509 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 
2023-01-18 12:08:41,526	INFO packaging.py:498 -- Creating a file package for local directory '..'.
2023-01-18 12:08:41,560	WARNING packaging.py:385 -- File /home/elad/docker/beamds/notebooks/../notebooks/stl10.ipynb is very large (12.18MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/elad/docker/beamds/notebooks/../notebooks/stl10.ipynb']})`
2023-01-18 12:08:41,586	WARNING packaging.py:385 -- File /home/elad/docker/beamds/notebooks/../notebooks/stl10-Copy1.ipynb is very large (12.61MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/elad/docker/beamds/notebooks/../notebooks/stl10-Copy1.ipynb']})`
2023-01-18 12:0

(scheduler +3s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +3s) Warning: The following resource request cannot be scheduled right now: {'CPU': 4.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(func_wrapper pid=7072) 2023-01-18 12:08:45 | INFO | Starting task: 0
(func_wrapper pid=7071) 2023-01-18 12:08:46 | INFO | Starting task: 0
(func_wrapper pid=7072) 2023-01-18 12:08:56 | INFO | Finished task: 0. Elapsed time: 10.348930358886719
(func_wrapper pid=7072) 2023-01-18 12:08:56 | INFO | Starting task: 0
(func_wrapper pid=7071) 2023-01-18 12:08:56 | INFO | Finished task: 0. Elapsed time: 10.397742509841919
(func_wrapper pid=7071) 2023-01-18 12:08:56 | INFO | Starting task: 0
(func_wrapper pid=7072) 2023-01-18 12:09:06 | INFO | Finished task: 0. Elapsed time: 10.397160530090332
(func_wrapper pid=7072) 2023-01-18 12:0

In [5]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='apply_async', workers=4, context='fork', reduce=True)

2023-01-18 12:06:21 | INFO | Starting task: 0
2023-01-18 12:06:21 | INFO | Starting task: 1
2023-01-18 12:06:21 | INFO | Starting task: 2
2023-01-18 12:06:21 | INFO | Starting task: 3
2023-01-18 12:06:31 | INFO | Finished task: 0. Elapsed time: 10.357866764068604
2023-01-18 12:06:31 | INFO | Starting task: 4


  5%|5         | 1/20 [00:00<?, ?it/s]

2023-01-18 12:06:31 | INFO | Finished task: 1. Elapsed time: 10.365689277648926
2023-01-18 12:06:31 | INFO | Finished task: 3. Elapsed time: 10.349919557571411
2023-01-18 12:06:31 | INFO | Starting task: 5
2023-01-18 12:06:31 | INFO | Starting task: 6
2023-01-18 12:06:31 | INFO | Finished task: 2. Elapsed time: 10.384727716445923
2023-01-18 12:06:31 | INFO | Starting task: 7
2023-01-18 12:06:42 | INFO | Finished task: 4. Elapsed time: 10.504316806793213
2023-01-18 12:06:42 | INFO | Starting task: 8
2023-01-18 12:06:42 | INFO | Finished task: 5. Elapsed time: 10.495826244354248
2023-01-18 12:06:42 | INFO | Finished task: 6. Elapsed time: 10.487637281417847
2023-01-18 12:06:42 | INFO | Starting task: 9
2023-01-18 12:06:42 | INFO | Starting task: 10
2023-01-18 12:06:42 | INFO | Finished task: 7. Elapsed time: 10.512568950653076
2023-01-18 12:06:42 | INFO | Starting task: 11
2023-01-18 12:06:52 | INFO | Finished task: 8. Elapsed time: 10.335124731063843
2023-01-18 12:06:52 | INFO | Startin

In [31]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='ray', workers=4, reduce=True, runtime_env={"working_dir": ".." })

2023-01-18 10:28:47,438	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://172.17.0.2:8265 
2023-01-18 10:28:47,657	INFO packaging.py:498 -- Creating a file package for local directory '..'.
2023-01-18 10:28:47,693	WARNING packaging.py:385 -- File /home/elad/docker/beamds/notebooks/../notebooks/stl10.ipynb is very large (12.18MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/elad/docker/beamds/notebooks/../notebooks/stl10.ipynb']})`
2023-01-18 10:28:47,719	WARNING packaging.py:385 -- File /home/elad/docker/beamds/notebooks/../notebooks/stl10-Copy1.ipynb is very large (12.61MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/elad/docker/beamds/notebooks/../notebooks/stl10-Copy1.ipynb']})`
2023-01-18 10:28:47,746	WARNING packaging.py:385 -- File /home/elad/docker/beamds/notebooks/../notebooks/beam_deep_tsne.ipyn

(func_wrapper pid=66872) 2023-01-18 10:28:53 | INFO | Starting task: 7
(func_wrapper pid=66879) 2023-01-18 10:28:53 | INFO | Starting task: 14
(func_wrapper pid=66876) 2023-01-18 10:28:53 | INFO | Starting task: 11
(func_wrapper pid=66881) 2023-01-18 10:28:53 | INFO | Starting task: 16
(func_wrapper pid=66868) 2023-01-18 10:28:53 | INFO | Starting task: 3
(func_wrapper pid=66884) 2023-01-18 10:28:53 | INFO | Starting task: 19
(func_wrapper pid=66867) 2023-01-18 10:28:53 | INFO | Starting task: 2
(func_wrapper pid=66880) 2023-01-18 10:28:53 | INFO | Starting task: 15
(func_wrapper pid=66877) 2023-01-18 10:28:53 | INFO | Starting task: 12
(func_wrapper pid=66875) 2023-01-18 10:28:53 | INFO | Starting task: 10
(func_wrapper pid=66883) 2023-01-18 10:28:53 | INFO | Starting task: 18
(func_wrapper pid=66878) 2023-01-18 10:28:53 | INFO | Starting task: 13
(func_wrapper pid=66873) 2023-01-18 10:28:53 | INFO | Starting task: 8
(func_wrapper pid=66865) 2023-01-18 10:28:53 | INFO | Starting task:

In [32]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='ray', workers=4, reduce=True, runtime_env={"working_dir": ".." })

2023-01-18 10:29:04,606	INFO worker.py:1351 -- Calling ray.init() again after it has already been called.


(func_wrapper pid=66869) 2023-01-18 10:29:04 | INFO | Finished task: 4. Elapsed time: 10.394327878952026
(func_wrapper pid=66868) 2023-01-18 10:29:04 | INFO | Starting task: 9
(func_wrapper pid=66867) 2023-01-18 10:29:04 | INFO | Starting task: 7
(func_wrapper pid=66880) 2023-01-18 10:29:04 | INFO | Starting task: 2
(func_wrapper pid=66878) 2023-01-18 10:29:04 | INFO | Starting task: 5
(func_wrapper pid=66871) 2023-01-18 10:29:04 | INFO | Starting task: 4
(func_wrapper pid=66869) 2023-01-18 10:29:04 | INFO | Starting task: 0
(func_wrapper pid=66865) 2023-01-18 10:29:04 | INFO | Starting task: 8
(func_wrapper pid=66870) 2023-01-18 10:29:04 | INFO | Starting task: 3
(func_wrapper pid=66866) 2023-01-18 10:29:04 | INFO | Starting task: 1
(func_wrapper pid=66874) 2023-01-18 10:29:04 | INFO | Starting task: 6
(func_wrapper pid=66881) 2023-01-18 10:29:04 | INFO | Starting task: 10
(func_wrapper pid=66877) 2023-01-18 10:29:04 | INFO | Starting task: 11
(func_wrapper pid=66883) 2023-01-18 10:29

In [30]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='apply_async', workers=4, context='fork', reduce=True)

2023-01-18 10:27:52 | INFO | Starting task: 0
2023-01-18 10:27:52 | INFO | Starting task: 1
2023-01-18 10:27:52 | INFO | Starting task: 2
2023-01-18 10:27:52 | INFO | Starting task: 3
2023-01-18 10:28:02 | INFO | Finished task: 3. Elapsed time: 10.340762376785278
2023-01-18 10:28:02 | INFO | Finished task: 0. Elapsed time: 10.365068674087524
2023-01-18 10:28:03 | INFO | Finished task: 2. Elapsed time: 10.376482725143433
2023-01-18 10:28:03 | INFO | Starting task: 4
2023-01-18 10:28:03 | INFO | Finished task: 1. Elapsed time: 10.36663007736206
2023-01-18 10:28:03 | INFO | Starting task: 5


  5%|5         | 1/20 [00:00<?, ?it/s]

2023-01-18 10:28:03 | INFO | Starting task: 6
2023-01-18 10:28:03 | INFO | Starting task: 7
2023-01-18 10:28:13 | INFO | Finished task: 4. Elapsed time: 10.349640369415283
2023-01-18 10:28:13 | INFO | Starting task: 8
2023-01-18 10:28:13 | INFO | Finished task: 5. Elapsed time: 10.35445499420166
2023-01-18 10:28:13 | INFO | Finished task: 7. Elapsed time: 10.33534550666809
2023-01-18 10:28:13 | INFO | Starting task: 9
2023-01-18 10:28:13 | INFO | Starting task: 10
2023-01-18 10:28:13 | INFO | Finished task: 6. Elapsed time: 10.385774612426758
2023-01-18 10:28:13 | INFO | Starting task: 11
2023-01-18 10:28:23 | INFO | Finished task: 8. Elapsed time: 10.349951982498169
2023-01-18 10:28:23 | INFO | Starting task: 12
2023-01-18 10:28:23 | INFO | Finished task: 9. Elapsed time: 10.353824138641357
2023-01-18 10:28:23 | INFO | Finished task: 10. Elapsed time: 10.349895000457764
2023-01-18 10:28:23 | INFO | Starting task: 13
2023-01-18 10:28:23 | INFO | Starting task: 14
2023-01-18 10:28:23 | 

In [29]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='dask', workers=4, reduce=True)

2023-01-18 10:26:49 | INFO | Starting task: 6
2023-01-18 10:26:49 | INFO | Starting task: 1
2023-01-18 10:26:49 | INFO | Starting task: 19
2023-01-18 10:26:49 | INFO | Starting task: 4
2023-01-18 10:27:02 | INFO | Finished task: 1. Elapsed time: 12.615652322769165
2023-01-18 10:27:02 | INFO | Finished task: 19. Elapsed time: 12.618026494979858
2023-01-18 10:27:02 | INFO | Starting task: 5
2023-01-18 10:27:02 | INFO | Starting task: 7
2023-01-18 10:27:02 | INFO | Finished task: 4. Elapsed time: 12.627742052078247
2023-01-18 10:27:02 | INFO | Finished task: 6. Elapsed time: 12.638269424438477
2023-01-18 10:27:02 | INFO | Starting task: 8
2023-01-18 10:27:02 | INFO | Starting task: 0
2023-01-18 10:27:14 | INFO | Finished task: 7. Elapsed time: 12.405572175979614
2023-01-18 10:27:14 | INFO | Starting task: 3
2023-01-18 10:27:14 | INFO | Finished task: 8. Elapsed time: 12.406249284744263
2023-01-18 10:27:14 | INFO | Starting task: 2
2023-01-18 10:27:14 | INFO | Finished task: 5. Elapsed tim

In [27]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='joblib', verbose=10, workers=4, reduce=True)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


2023-01-18 10:24:54 | INFO | Starting task: 0
2023-01-18 10:24:55 | INFO | Starting task: 3
2023-01-18 10:24:55 | INFO | Starting task: 1
2023-01-18 10:24:55 | INFO | Starting task: 2
2023-01-18 10:25:05 | INFO | Finished task: 0. Elapsed time: 10.42505431175232
2023-01-18 10:25:05 | INFO | Finished task: 3. Elapsed time: 10.37529730796814
2023-01-18 10:25:05 | INFO | Finished task: 1. Elapsed time: 10.386337280273438
2023-01-18 10:25:05 | INFO | Finished task: 2. Elapsed time: 10.350576162338257
2023-01-18 10:25:05 | INFO | Starting task: 4
2023-01-18 10:25:05 | INFO | Starting task: 5
2023-01-18 10:25:05 | INFO | Starting task: 6
2023-01-18 10:25:05 | INFO | Starting task: 7
2023-01-18 10:25:15 | INFO | Finished task: 4. Elapsed time: 10.350367784500122
2023-01-18 10:25:15 | INFO | Starting task: 8
2023-01-18 10:25:15 | INFO | Finished task: 5. Elapsed time: 10.34516429901123
2023-01-18 10:25:15 | INFO | Starting task: 9
2023-01-18 10:25:15 | INFO | Finished task: 6. Elapsed time: 10

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   23.5s


2023-01-18 10:25:26 | INFO | Finished task: 8. Elapsed time: 10.341449499130249
2023-01-18 10:25:26 | INFO | Starting task: 12
2023-01-18 10:25:26 | INFO | Finished task: 9. Elapsed time: 10.34509801864624
2023-01-18 10:25:26 | INFO | Starting task: 13
2023-01-18 10:25:26 | INFO | Finished task: 10. Elapsed time: 10.34015703201294
2023-01-18 10:25:26 | INFO | Starting task: 14
2023-01-18 10:25:26 | INFO | Finished task: 11. Elapsed time: 10.336136102676392
2023-01-18 10:25:26 | INFO | Starting task: 15


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   33.8s


2023-01-18 10:25:36 | INFO | Finished task: 12. Elapsed time: 10.34852647781372
2023-01-18 10:25:36 | INFO | Starting task: 16
2023-01-18 10:25:36 | INFO | Finished task: 13. Elapsed time: 10.347316265106201
2023-01-18 10:25:36 | INFO | Starting task: 17
2023-01-18 10:25:36 | INFO | Finished task: 14. Elapsed time: 10.364957571029663
2023-01-18 10:25:36 | INFO | Starting task: 18
2023-01-18 10:25:36 | INFO | Finished task: 15. Elapsed time: 10.351532220840454
2023-01-18 10:25:36 | INFO | Starting task: 19


[Parallel(n_jobs=4)]: Done  16 out of  20 | elapsed:   44.2s remaining:   11.1s


2023-01-18 10:25:46 | INFO | Finished task: 17. Elapsed time: 10.35016417503357
2023-01-18 10:25:46 | INFO | Finished task: 16. Elapsed time: 10.403868913650513
2023-01-18 10:25:46 | INFO | Finished task: 18. Elapsed time: 10.356250286102295
2023-01-18 10:25:47 | INFO | Finished task: 19. Elapsed time: 10.358731508255005
CPU times: user 163 ms, sys: 176 ms, total: 339 ms
Wall time: 54.7 s


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:   54.6s finished


In [24]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='process_map', func=func, workers=4, reduce=True)

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: user 130 ms, sys: 154 ms, total: 284 ms
Wall time: 51.9 s


In [28]:
%time dfo = parallel([task(func)(dfi) for _, dfi in dfs], method='thread_map', func=func, workers=4, reduce=True)

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: user 12.6 s, sys: 384 ms, total: 13 s
Wall time: 1min 2s


In [17]:
# bp = BeamParallel(method='apply_async', workers=4, context='fork', reduce=True)
bp = BeamParallel(method='dask', workers=4, reduce=True)

In [ ]:
bp = BeamParallel(method='apply_async', workers=4, context='fork', reduce=True)

In [18]:
for _, dfi in dfs:
    bp.add(func, dfi)

In [19]:
dfo = bp()

2023-01-17 13:43:17 | INFO | Starting task: 7
2023-01-17 13:43:17 | INFO | Starting task: 5
2023-01-17 13:43:17 | INFO | Starting task: 9
2023-01-17 13:43:17 | INFO | Starting task: 11
2023-01-17 13:43:17 | INFO | Starting task: 8
2023-01-17 13:43:17 | INFO | Starting task: 16
2023-01-17 13:43:17 | INFO | Starting task: 17
2023-01-17 13:43:17 | INFO | Starting task: 19
2023-01-17 13:43:17 | INFO | Starting task: 14
2023-01-17 13:43:17 | INFO | Starting task: 15
2023-01-17 13:43:17 | INFO | Starting task: 13
2023-01-17 13:43:17 | INFO | Starting task: 12
2023-01-17 13:43:17 | INFO | Starting task: 10
2023-01-17 13:43:17 | INFO | Starting task: 6
2023-01-17 13:43:17 | INFO | Starting task: 1
2023-01-17 13:43:17 | INFO | Starting task: 2
2023-01-17 13:43:17 | INFO | Starting task: 18
2023-01-17 13:43:17 | INFO | Starting task: 4
2023-01-17 13:43:17 | INFO | Starting task: 3
2023-01-17 13:43:17 | INFO | Starting task: 0
2023-01-17 13:43:41 | INFO | Finished task: 19. Elapsed time: 23.91254

In [9]:
res = _7

In [13]:
with BeamParallel(method='apply_async', workers=4, context='fork', reduce=True) as bp:
    for dfi in dfs:
        bp.add(func, df)

2023-01-17 13:15:35 | INFO | Starting task: 0
2023-01-17 13:15:35 | INFO | Starting task: 1
2023-01-17 13:15:35 | INFO | Starting task: 2
2023-01-17 13:15:35 | INFO | Starting task: 3
2023-01-17 13:15:36 | INFO | Finished task: 1. Elapsed time: 0.5444965362548828
2023-01-17 13:15:36 | INFO | Starting task: 4
2023-01-17 13:15:36 | INFO | Finished task: 3. Elapsed time: 0.5805377960205078
2023-01-17 13:15:36 | INFO | Finished task: 2. Elapsed time: 0.6087396144866943
2023-01-17 13:15:36 | INFO | Starting task: 5
2023-01-17 13:15:36 | INFO | Starting task: 6
2023-01-17 13:15:36 | INFO | Finished task: 0. Elapsed time: 0.6630456447601318
2023-01-17 13:15:36 | INFO | Starting task: 7
2023-01-17 13:15:37 | INFO | Finished task: 4. Elapsed time: 0.531965970993042
2023-01-17 13:15:37 | INFO | Starting task: 8
2023-01-17 13:15:37 | INFO | Finished task: 6. Elapsed time: 0.5288615226745605
2023-01-17 13:15:37 | INFO | Finished task: 5. Elapsed time: 0.5395429134368896
2023-01-17 13:15:37 | INFO 

In [7]:
bp()

2023-01-17 13:10:22 | INFO | Starting task: 13
2023-01-17 13:10:22 | INFO | Starting task: 4
2023-01-17 13:10:22 | INFO | Starting task: 1
2023-01-17 13:10:22 | INFO | Starting task: 8
2023-01-17 13:10:22 | INFO | Starting task: 7
2023-01-17 13:10:22 | INFO | Starting task: 15
2023-01-17 13:10:22 | INFO | Starting task: 9
2023-01-17 13:10:22 | INFO | Starting task: 0
2023-01-17 13:10:23 | INFO | Starting task: 17
2023-01-17 13:10:23 | INFO | Starting task: 19
2023-01-17 13:10:23 | INFO | Starting task: 16
2023-01-17 13:10:23 | INFO | Starting task: 10
2023-01-17 13:10:23 | INFO | Starting task: 11
2023-01-17 13:10:23 | INFO | Starting task: 3
2023-01-17 13:10:23 | INFO | Starting task: 2
2023-01-17 13:10:23 | INFO | Starting task: 5
2023-01-17 13:10:23 | INFO | Starting task: 12
2023-01-17 13:10:24 | INFO | Starting task: 6
2023-01-17 13:10:24 | INFO | Starting task: 18
2023-01-17 13:10:24 | INFO | Starting task: 14
2023-01-17 13:10:48 | INFO | Finished task: 15. Elapsed time: 24.61581

[[           sum      mean
  0   -27.597440 -0.055195
  1     2.977033  0.005954
  2     3.296336  0.006593
  3    -0.431653 -0.000863
  4   -11.197707 -0.022395
  ..         ...       ...
  995  20.365816  0.040732
  996  -6.227476 -0.012455
  997 -34.546115 -0.069092
  998   2.362296  0.004725
  999  -2.332649 -0.004665
  
  [1000 rows x 2 columns],
             sum      mean
  0   -27.597440 -0.055195
  1     2.977033  0.005954
  2     3.296336  0.006593
  3    -0.431653 -0.000863
  4   -11.197707 -0.022395
  ..         ...       ...
  995  20.365816  0.040732
  996  -6.227476 -0.012455
  997 -34.546115 -0.069092
  998   2.362296  0.004725
  999  -2.332649 -0.004665
  
  [1000 rows x 2 columns],
             sum      mean
  0   -27.597440 -0.055195
  1     2.977033  0.005954
  2     3.296336  0.006593
  3    -0.431653 -0.000863
  4   -11.197707 -0.022395
  ..         ...       ...
  995  20.365816  0.040732
  996  -6.227476 -0.012455
  997 -34.546115 -0.069092
  998   2.362296  0.00

In [11]:
res = parallelize(func, dfs, method='process_map')

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
res = parallelize(func, dfs, workers=0, method='process_map')

AssertionError: divide_chunks supports only array types

In [ ]:
res[0].get()

In [13]:
tuple(2 * i for i in range(10))

(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)

In [9]:
res.

In [8]:
res.get()

KeyboardInterrupt: 

In [20]:
import itertools
from beam import check_type

In [14]:
ii = itertools.count()

In [16]:
def divide_chunks(x, chunksize=None, n_chunks=None, dim=0):

    assert ((chunksize is None) != (n_chunks is None)), "divide_chunks requires only one of chunksize|n_chunks"
    x_type = check_type(x, check_element=False)
    assert x_type.major in ['array', 'other'], "divide_chunks supports only array types"

    if x_type.major == 'array':

        l = len(x)

        if chunksize is None:
            chunksize = l // n_chunks

        if n_chunks is None:
            n_chunks = int(np.round(l / chunksize))

        if x_type.minor == 'tensor':
            for c in torch.chunk(x, n_chunks, dim=dim):
                yield c

        elif x_type.minor in ['numpy', 'pandas']:

            for c in np.array_split(x, n_chunks, axis=dim):
                yield c

        else:

            for i in range(0, l, chunksize):
                yield x[i:i + chunksize]

    else:

        c = []
        for xi in iter(x):

            c.append(xi)
            if len(c) == chunksize:
                yield c
                c = []

        yield c

In [62]:
def recursive_chunks(x, chunksize=None, n_chunks=None, dim=0):

    try:
        if isinstance(x, dict):
            gen = {k: recursive_chunks(v, chunksize=chunksize, n_chunks=n_chunks, dim=dim) for k, v in x.items()}
            for _ in itertools.count():
                yield {k: next(v) for k, v in gen.items()}
        elif isinstance(x, list) or isinstance(x, tuple):

            gen = [recursive_chunks(s, chunksize=chunksize, n_chunks=n_chunks, dim=dim) for s in x]
            for _ in itertools.count():
                yield [next(s) for s in gen]
        elif x is None:
            for _ in itertools.count():
                yield None
        else:
            for c in divide_chunks(x, chunksize=chunksize, n_chunks=n_chunks, dim=dim):
                yield c
            
    except StopIteration:
        return

In [63]:
x = {'x': torch.randn(1000, 30), 'y': torch.randn(1000, 30), 'z': None}

In [32]:
g = divide_chunks(x, chunksize=22)

In [64]:
g = recursive_chunks(x, chunksize=22)

In [65]:
l = list(g)

In [68]:
x = torch.randn(100, 32)

In [70]:
x[:, 5:8].shape

torch.Size([100, 3])

In [82]:
np.array_split(np.arange(20), 6)[0]

array([0, 1, 2, 3])

In [ ]:
np.array_split(np.arange(20), 6)[0]

In [80]:
[c.tolist() for c in np.array_split([1, 2, 'a', 4, 5, 6, 7, 8, 9, 10], 6)]

[['1', '2'], ['a', '4'], ['5', '6'], ['7', '8'], ['9'], ['10']]

In [75]:
20 // 3

6

In [77]:
torch.chunk(torch.arange(20), 6)

(tensor([0, 1, 2, 3]),
 tensor([4, 5, 6, 7]),
 tensor([ 8,  9, 10, 11]),
 tensor([12, 13, 14, 15]),
 tensor([16, 17, 18, 19]))

In [113]:
df = pd.DataFrame({'a': [1, np.pi], 'b': ['3', 4]})

In [112]:
df.to_pickle('/tmp/df.fea')

In [104]:
df = pd.DataFrame({'a': [1, 2], 'b': [3, 4], 'c': [[1,2,3], [4,5,6]]})

In [95]:
df['b'] = df['b'].astype('int')

In [106]:
df.to_feather('/tmp/df.p')

In [107]:
df.to_parquet('/tmp/df.p')

In [109]:
df.dtypes['c']

dtype('O')

In [117]:
from collections import OrderedDict, defaultdict

In [118]:
d = defaultdict(list)

In [119]:
isinstance(d, dict)

True

In [120]:
a = torch.randn(10000)

In [122]:
df

,a,b
0,1.000000,3
1,3.141593,4


In [131]:
import sys

In [133]:
sys.getsizeof(['a', 'f'])

72